<a href="https://colab.research.google.com/github/RoboTuan/ML_DL_Project/blob/master/Ges%C3%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title
import torch
import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn
import tensorflow as tf
import datetime
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
from torch.autograd import Function
import torchvision
from torchvision.models import alexnet
from torchvision.utils import make_grid
from sklearn.model_selection import RandomizedSearchCV
import pandas.util.testing as tm
from PIL import Image
from tqdm.notebook import tqdm

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -q "./drive/My Drive/Copia di GTEA61.zip"
!rm -rf "./__MACOSX"

In [20]:
!rm -rf "./ML_DL_Project/"
#!rm -rf "./out_dir/"

In [2]:
import os 
if not os.path.isdir('./ML_DL_Project'):
  !git clone https://github.com/RoboTuan/ML_DL_Project.git

Cloning into 'ML_DL_Project'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 340 (delta 74), reused 60 (delta 32), pack-reused 230
Receiving objects: 100% (340/340), 89.23 MiB | 13.85 MiB/s, done.
Resolving deltas: 100% (208/208), done.


In [ ]:
base_dataset = "./GTEA61"

if not os.path.isdir('./GTEA61_val'):
  os.makedirs("./GTEA61_val")
  os.makedirs("./GTEA61_val/flow_x_processed")
  os.makedirs("./GTEA61_val/flow_y_processed")
  os.makedirs("./GTEA61_val/processed_frames2")
  pass


for directory in sorted(os.listdir(base_dataset)):
  if not directory.startswith('.'):
    directory_path = os.path.join(base_dataset, directory)
    new_directory_path = os.path.join("./GTEA61_val", directory)

    for user in sorted(os.listdir(directory_path)):
      if not user.startswith('.') and user == "S2":
        current_folder = os.path.join(directory_path, user)
        !cp -r "{current_folder}" "{new_directory_path}"
        !rm -rf "{current_folder}"
        print(current_folder)

./GTEA61/flow_x_processed/S2
./GTEA61/flow_y_processed/S2
./GTEA61/processed_frames2/S2


In [3]:
# Se scritti uguali ai parametri della funzione può venire None come risultato
# per questo SEQLEN è maiuscolo 
SEQLEN = 7
DEVICE = 'cuda' # 'cuda' or 'cpu'
num_classes = 61
#STAGE = None
train_data_dir = "./GTEA61"
val_data_dir = "./GTEA61_val"
#stage1_dict = None
out_dir = ".out_dir"
trainBatchSize = 32
valBatchSize = 64
numEpochs1 = 3
numEpochs2 = 3
lr1 = 1e-3
lr2 = 1e-4
decay_factor = 0.1
decay_step1 = [25, 75, 150]
decay_step2 = [25, 75]
#Boh?!
memSize = 512
alpha=1

In [12]:
#!rm -rf "ML_DL_Project"
#!rm -rf "./GTEA61"
#!rm -rf "./GTEA61_val"

In [4]:
from __future__ import print_function, division
# This is without attention, we must address this better
#from ML_DL_Project.Scripts.convLSTMmodel import *
from ML_DL_Project.Scripts.SelfSupObjectAttentionModelConvLSTM import *
from ML_DL_Project.Scripts.RegObjectAttentionModelConvLSTM import *
from ML_DL_Project.Scripts.spatial_transforms import (Compose, ToTensor, CenterCrop, Scale, Normalize, MultiScaleCornerCrop,
                                RandomHorizontalFlip)
from torch.utils.tensorboard import SummaryWriter
from ML_DL_Project.Scripts.resnetMod import *
#from ML_DL_Project.Scripts.makeDatasetRGB import *
from ML_DL_Project.Scripts.makeMmaps import *
#Prende il makeDataset dell'ultimo script importato
import argparse
import sys

In [ ]:
!rm -rf "./out_dir"

In [5]:
# Data loader
normalize = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
spatial_transform = Compose([Scale(256),
                             RandomHorizontalFlip(), 
                             MultiScaleCornerCrop([1, 0.875, 0.75, 0.65625], 224),
                             ToTensor(), 
                             normalize])

vid_seq_train = makeDataset(train_data_dir, spatial_transform=spatial_transform, seqLen=SEQLEN, fmt='.png')

train_loader = torch.utils.data.DataLoader(vid_seq_train, batch_size=trainBatchSize, shuffle=True, num_workers=4, pin_memory=True)

if val_data_dir is not None:
  vid_seq_val = makeDataset(val_data_dir,spatial_transform = Compose([Scale(256),
                                                                    CenterCrop(224),
                                                                    ToTensor(),
                                                                    normalize]),
                            seqLen=SEQLEN, fmt='.png')

  val_loader = torch.utils.data.DataLoader(vid_seq_val, batch_size=valBatchSize, shuffle=False, num_workers=2, pin_memory=True)
  valInstances = vid_seq_val.__len__()


trainInstances = vid_seq_train.__len__()

In [ ]:
for seqImages, seqMaps, labels in train_loader:
  print(f"Batch size: {len(seqImages)}")
  #Make dataset restituisce una sequenza di immagini e di mappe
  print(f"Batch size: {len(maps)}")
  print(f"Number of frames selected: {len(seqImages[0])}")
  print(f"Number of frames selected maps: {len(seqMaps[0])}")

  break

Batch size: 32
Batch size: 32
Number of frames selected: 7
Number of frames selected maps: 7


In [6]:
import torch
import ML_DL_Project.Scripts.resnetMod
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable
from ML_DL_Project.Scripts.MyConvLSTMCell import *

class Print(nn.Module):
    def forward(self, x):
        print(x.size())
        return x

class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

class attentionModel(nn.Module):
    def __init__(self, num_classes=61, mem_size=512):
        super(attentionModel, self).__init__()
        self.num_classes = num_classes
        self.resNet = resnetMod.resnet34(True, True)
        self.mem_size = mem_size
        self.weight_softmax = self.resNet.fc.weight
        self.lstm_cell = MyConvLSTMCell(512, mem_size)
        self.avgpool = nn.AvgPool2d(7)
        self.dropout = nn.Dropout(0.7)
        self.fc = nn.Linear(mem_size, self.num_classes)
        self.classifier = nn.Sequential(self.dropout, self.fc)


        #Secondary task branch
        self.mmapPredictor = nn.Sequential()
        self.mmapPredictor.add_module('mmap_relu',nn.ReLU(True))
        self.mmapPredictor.add_module('convolution', nn.Conv2d(512, 100, kernel_size=1))
        self.mmapPredictor.add_module('flatten',Flatten())
        self.mmapPredictor.add_module('fc_2',nn.Linear(100*7*7,2*7*7))



    def forward(self, inputVariable):
        state = (Variable(torch.zeros((inputVariable.size(1), self.mem_size, 7, 7)).cuda()),
                 Variable(torch.zeros((inputVariable.size(1), self.mem_size, 7, 7)).cuda()))
        
        for t in range(inputVariable.size(0)):
            logit, feature_conv, feature_convNBN = self.resNet(inputVariable[t])
            bz, nc, h, w = feature_conv.size()
            feature_conv1 = feature_conv.view(bz, nc, h*w)
            
            probs, idxs = logit.sort(1, True)
            class_idx = idxs[:, 0]
            cam = torch.bmm(self.weight_softmax[class_idx].unsqueeze(1), feature_conv1)
            
            attentionMAP = F.softmax(cam.squeeze(1), dim=1)
            attentionMAP = attentionMAP.view(attentionMAP.size(0), 1, 7, 7)
            attentionFeat = feature_convNBN * attentionMAP.expand_as(feature_conv)

            #Prediction of the mmap
            feature_conv2 = feature_conv.clone()
            if t == 0:
              map_predictions = self.mmapPredictor(feature_conv2)
            else:
              prediction = self.mmapPredictor(feature_conv2) #This can be feature_conv
              map_predictions = torch.cat([map_predictions,prediction],dim=0)

            state = self.lstm_cell(attentionFeat, state)
        
        feats1 = self.avgpool(state[1]).view(state[1].size(0), -1)
        feats = self.classifier(feats1)
        return feats, feats1, map_predictions#Makes the list a stack

In [7]:
train_params = []
model = attentionModel(num_classes=num_classes, mem_size=memSize)
model.train(False)
for params in model.parameters():
    params.requires_grad = False

for params in model.lstm_cell.parameters():
    params.requires_grad = True
    train_params += [params]

for params in model.classifier.parameters():
    params.requires_grad = True
    train_params += [params]

model.lstm_cell.train(True)

model.classifier.train(True)
model.cuda()

loss_fn = nn.CrossEntropyLoss()

optimizer_fn = torch.optim.Adam(train_params, lr=lr1, weight_decay=4e-5, eps=1e-4)

optim_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer_fn, milestones=decay_step1,
                                                           gamma=decay_factor)



In [8]:
model

attentionModel(
  (resNet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tra

In [11]:
valDatasetDir = val_data_dir

train_iter = 0
min_accuracy = 0
for epoch in range(numEpochs1):
    epoch_loss = 0
    numCorrTrain = 0
    mmap_loss = 0
    trainSamples = 0
    iterPerEpoch = 0
    
    model.lstm_cell.train(True)
    model.classifier.train(True)
    
    
    #for i, (inputs, targets) in enumerate(train_loader):
    for inputs, inputMmap, targets in train_loader:
        train_iter += 1
        iterPerEpoch += 1
        optimizer_fn.zero_grad()

        inputMmap = inputMmap.cuda()
        
        inputVariable = Variable(inputs.permute(1, 0, 2, 3, 4).cuda())
        labelVariable = Variable(targets.cuda())
        trainSamples += inputs.size(0)
        
        output_label, _ , mmapPrediction = model(inputVariable)

        mmapPrediction = mmapPrediction.view(-1,2)         
        inputMmap = torch.reshape(inputMmap, (-1,)) #.long()
        inputMmap = torch.round(inputMmap).long() #making things black and white again
        
        
        #print(mmapPrediction.size())
        #print(inputMmap.size())
        #print(mmapPrediction)
        #print(inputMmap)
        #sys.exit()

        loss2 = alpha*loss_fn(mmapPrediction,inputMmap)
        loss = loss_fn(output_label, labelVariable)

        #Weighting the loss of the ss task
        #by multiplying it by alpha
        total_loss = loss  + loss2
        total_loss.backward()
        
        optimizer_fn.step()
        _, predicted = torch.max(output_label.data, 1)
        numCorrTrain += (predicted == targets.cuda()).sum()
        mmap_loss += loss2.item()
        epoch_loss += loss.item()
    
    optim_scheduler.step()
    avg_mmap_loss = mmap_loss / iterPerEpoch
    avg_loss = epoch_loss/iterPerEpoch
    trainAccuracy = torch.true_divide(numCorrTrain , trainSamples) * 100

    print('Train: Epoch = {} | Loss = {} | Accuracy = {}'.format(epoch+1, avg_loss, trainAccuracy))
    print('Mmap loss after {} epoch = {}% '.format(epoch + 1, avg_mmap_loss))

    
    if valDatasetDir is not None:
        model.train(False)
        val_loss_epoch = 0
        val_iter = 0
        val_mmap_loss = 0
        val_samples = 0
        numCorr = 0
        mmap_loss = 0

        with torch.no_grad():
            #for j, (inputs, targets) in enumerate(val_loader):
            for inputs, inputMmap, targets in val_loader:
                val_iter += 1
                val_samples += inputs.size(0)
                inputMmap = inputMmap.cuda()

                
                inputVariable = Variable(inputs.permute(1, 0, 2, 3, 4).cuda())
                labelVariable = Variable(targets.cuda(async=True))
                #labelVariable = Variable(targets.cuda())
                
                output_label, _ , mmapPrediction = model(inputVariable)

                mmapPrediction = mmapPrediction.view(-1,2)
                inputMmap = torch.reshape(inputMmap, (-1,)) #.long()
                inputMmap = torch.round(inputMmap).long()
                loss2 = alpha*loss_fn(mmapPrediction,inputMmap)

                val_loss = loss_fn(output_label, labelVariable)
                val_loss_epoch += val_loss.item()
                val_mmap_loss += loss2.item()
                
                _, predicted = torch.max(output_label.data, 1)
                numCorr += (predicted == targets.cuda()).sum()

        avg_mmap_val_loss = val_mmap_loss / val_iter
        val_accuracy = torch.true_divide(numCorr , val_samples) * 100
        avg_val_loss = val_loss_epoch / val_iter
        
        print('Val MMap Loss after {} epochs, loss = {}'.format(epoch + 1, avg_mmap_val_loss))
        print('Val: Epoch = {} | Loss {} | Accuracy = {}'.format(epoch + 1, avg_val_loss, val_accuracy))

        #if val_accuracy > min_accuracy:
            #save_path_model = (model_folder + '/model_rgb_state_dict.pth')
            #torch.save(model.state_dict(), save_path_model)
            #min_accuracy = val_accuracy


torch.Size([10976, 2])
torch.Size([10976])
tensor([[-0.2914, -0.4953],
        [-0.0058,  0.1116],
        [ 0.3728,  0.1879],
        ...,
        [-0.0748, -0.4045],
        [ 0.7894,  0.3672],
        [-0.1829, -0.2505]], device='cuda:0')
tensor([0, 0, 0,  ..., 0, 0, 0], device='cuda:0')
torch.Size([10976, 2])
torch.Size([10976])
tensor([[-0.1900, -0.6978],
        [ 0.6372,  0.5620],
        [ 0.0993, -0.6612],
        ...,
        [-0.0764, -0.0697],
        [ 0.6777,  0.1872],
        [ 0.0274, -0.3665]], device='cuda:0')
tensor([0, 0, 0,  ..., 0, 0, 0], device='cuda:0')
torch.Size([10976, 2])
torch.Size([10976])
tensor([[ 0.1531,  0.0956],
        [-0.6522, -0.4256],
        [-0.0981,  0.7898],
        ...,
        [-0.5249,  0.4968],
        [ 0.7937,  0.5105],
        [ 0.0744,  0.2315]], device='cuda:0')
tensor([0, 0, 0,  ..., 0, 0, 0], device='cuda:0')
torch.Size([10976, 2])
torch.Size([10976])
tensor([[-0.5003,  0.3058],
        [ 0.8362, -0.0185],
        [-0.1586,  0.4534

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:82: UserWarning: This overload of cuda is deprecated:
	cuda(torch.device device, bool async, *, torch.memory_format memory_format)
Consider using one of the following signatures instead:
	cuda(torch.device device, bool non_blocking, *, torch.memory_format memory_format) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)


Val MMap Loss after 1 epochs, loss = 0.7899662554264069
Val: Epoch = 1 | Loss 3.902999758720398 | Accuracy = 6.034482479095459


NameError: ignored

In [ ]:
yolo = vid_seq_train.maps

In [ ]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
import random
import glob
import sys


from ML_DL_Project.Scripts.spatial_transforms import (Compose, ToTensor, CenterCrop, Scale, Normalize, MultiScaleCornerCrop,
                                RandomHorizontalFlip)

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
normalize = Normalize(mean=mean, std=std)
spatial_transform2 = Compose([Scale((7,7)), ToTensor()]) 

def listDirectory(path):
  if os.path.isdir(path):
    return os.listdir(path)
  
  return []

def gen_split(root_dir, stackSize):
    DatasetF = []
    Labels = []
    NumFrames = []
    #The root directory should be processed frames/train or test
    for dir_user in sorted(os.listdir(root_dir)):
        class_id = 0
        dir = os.path.join(root_dir, dir_user)
        for target in sorted(os.listdir(dir)):
            dir1 = os.path.join(dir, target)
            insts = sorted(listDirectory(dir1))              
            if insts != []:
                for inst in insts:
                    inst_dir = os.path.join(dir1,inst,'rgb')
                    inst_dir2 = os.path.join(dir1,inst)
                    numFrames = len(glob.glob1(inst_dir, '*.png'))

                    if numFrames >= stackSize:
                        DatasetF.append(inst_dir2)
                        Labels.append(class_id)
                        NumFrames.append(numFrames)
                class_id += 1
    bad_lab = [x for x in Labels if x>=61]
    #print(f'Bad labels {bad_lab}')
    return DatasetF, Labels, NumFrames


class makeDatasetPedro(Dataset):
    def __init__(self, root_dir, spatial_transform=None, sequence=False, stackSize=5,
                 train=True, numSeg=5, fmt='.png', phase='train', seqLen = 25):
        """
        Args:
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """

        self.imagesF, self.labels, self.numFrames = gen_split(root_dir, stackSize)
        self.spatial_transform = spatial_transform
        self.train = train
        self.numSeg = numSeg
        self.sequence = sequence
        self.stackSize = stackSize
        self.fmt = fmt
        self.phase = phase
        self.seqLen = seqLen

    def __len__(self):
        return len(self.imagesF)

    def __getitem__(self, idx):
        vid_nameF = self.imagesF[idx]
        label = self.labels[idx]
        numFrame = self.numFrames[idx]
        inpSeqSegs = []
        self.spatial_transform.randomize_parameters()

        inpSeqF = []
        for i in np.linspace(1, numFrame, self.seqLen, endpoint=False):
            fl_name = vid_nameF +  '/' + 'rgb' +'/' + 'rgb' + str(int(np.floor(i))).zfill(4) + self.fmt
            img = Image.open(fl_name)
            inpSeqF.append(self.spatial_transform(img.convert('RGB')))
        inpSeqF = torch.stack(inpSeqF, 0)

        #Adding the mmaps to the dataloader
        inpSeqMmaps = []
        try:
          inpSeqMmaps = []
          for i in np.linspace(1,numFrame, self.seqLen, endpoint=False):
            fl_name = vid_nameF +  '/' + 'mmaps' +'/' + 'map' + str(int(np.floor(i))).zfill(4) + self.fmt
            img = Image.open(fl_name)
            inpSeqMmaps.append(spatial_transform2(img.convert('L')))
        except:
            inpSeqMmaps = []
            for i in np.linspace(2,numFrame, self.seqLen, endpoint=False):
              fl_name = vid_nameF +  '/' + 'mmaps' +'/' + 'map' + str(int(np.floor(i))).zfill(4) + self.fmt
              img = Image.open(fl_name)
              inpSeqMmaps.append(spatial_transform2(img.convert('L'))) #Grayscale
  
        inpSeqMmaps = torch.stack(inpSeqMmaps,0)

        return inpSeqF, inpSeqMmaps ,label#, vid_nameF#, fl_name

In [ ]:
!find ./GTEA61 -name .DS_Store -type f -delete

In [ ]:
pedro = makeDatasetPedro("./GTEA61/processed_frames2", spatial_transform=spatial_transform, seqLen=SEQLEN, fmt='.png')

In [ ]:
pedro.imagesF

['./GTEA61/processed_frames2/S1/close_chocolate/1',
 './GTEA61/processed_frames2/S1/close_coffee/1',
 './GTEA61/processed_frames2/S1/close_coffee/2',
 './GTEA61/processed_frames2/S1/close_honey/1',
 './GTEA61/processed_frames2/S1/close_honey/2',
 './GTEA61/processed_frames2/S1/close_jam/1',
 './GTEA61/processed_frames2/S1/close_ketchup/1',
 './GTEA61/processed_frames2/S1/close_mayonnaise/1',
 './GTEA61/processed_frames2/S1/close_mustard/1',
 './GTEA61/processed_frames2/S1/close_mustard/2',
 './GTEA61/processed_frames2/S1/close_peanut/1',
 './GTEA61/processed_frames2/S1/close_peanut/2',
 './GTEA61/processed_frames2/S1/close_sugar/1',
 './GTEA61/processed_frames2/S1/close_sugar/2',
 './GTEA61/processed_frames2/S1/close_water/1',
 './GTEA61/processed_frames2/S1/close_water/2',
 './GTEA61/processed_frames2/S1/close_water/3',
 './GTEA61/processed_frames2/S1/fold_bread/1',
 './GTEA61/processed_frames2/S1/open_cheese/1',
 './GTEA61/processed_frames2/S1/open_chocolate/1',
 './GTEA61/processed_